In [1]:
import numpy as np
import pandas as pd
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import plotly.express as px
import plotly.offline as pyo
import seaborn as sns

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

# !pip install interpret
# from interpret.blackbox import LimeTabular
# from interpret import show

# import lime
# import lime.lime_tabular
# from lime.lime_tabular import LimeTabularExplainer
# import graphviz
# import shap

# import pickle

import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
network_data=pd.read_csv("/content/drive/MyDrive/CSE-CIC-IDS2018/new/cic-ids.csv")

In [4]:
network_data

,Dst Port,Protocol,Bwd Pkt Len Min,Flow Byts/s,Fwd IAT Mean,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,...,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Std,Active Max,Active Min,Idle Max,Idle Min,Label
0,445,6,0.0,0.000000,1.294750e+05,465.0,0.0,0.000000e+00,0.0,0.0,...,8192,8192,0,20,0.0,0.0,0.0,0.0,0.0,0
1,3389,6,0.0,1929.697409,1.839667e+05,31.0,1655539.0,3.167850e+05,953166.0,104244.0,...,8192,64000,7,20,0.0,0.0,0.0,0.0,0.0,0
2,10804,6,0.0,0.000000,4.600000e+01,46.0,0.0,0.000000e+00,0.0,0.0,...,258,-1,0,20,0.0,0.0,0.0,0.0,0.0,0
3,443,6,34.0,281.709029,1.974630e+06,15378.0,116487637.0,1.988388e+06,4014983.0,1019.0,...,255,440,29,20,0.0,0.0,0.0,0.0,0.0,0
4,56057,6,0.0,0.000000,4.100000e+01,41.0,0.0,0.000000e+00,0.0,0.0,...,257,-1,0,20,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905985,8080,6,0.0,42086.763480,2.690000e+02,55.0,10348.0,5.593371e+03,9905.0,51.0,...,8192,219,1,20,0.0,0.0,0.0,0.0,0.0,1
905986,8080,6,0.0,39045.739290,2.710000e+02,52.0,11183.0,6.153569e+03,10831.0,23.0,...,8192,219,1,20,0.0,0.0,0.0,0.0,0.0,1
905987,8080,6,0.0,0.000000,4.420000e+02,442.0,0.0,0.000000e+00,0.0,0.0,...,2052,-1,0,20,0.0,0.0,0.0,0.0,0.0,1
905988,8080,6,0.0,48424.861640,2.735000e+02,55.0,8926.0,4.774076e+03,8483.0,19.0,...,8192,219,1,20,0.0,0.0,0.0,0.0,0.0,1


**Load Dataset**

In [5]:
# #Save Data
# network_data.to_csv('/kaggle/working/cic-ids.csv', index=False)

In [6]:
%%time
# while working comment one line and uncomment another as needed
# temp_network_data = network_data
# network_data = temp_network_data

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


**Sequential Feature Selection**

In [7]:
%%time
# split data into features and target
X=network_data.drop(["Label"], axis=1)
y=network_data["Label"]

CPU times: user 38.6 ms, sys: 52.6 ms, total: 91.2 ms
Wall time: 105 ms


In [8]:
# # Import necessary libraries
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_selection import RFE
# from sklearn.metrics import classification_report
# import matplotlib.pyplot as plt



# # Initialize the Random Forest classifier
# rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# # Initialize RFE
# rfe = RFE(estimator=rf, n_features_to_select=10, step=1)  # You can adjust n_features_to_select

# # Fit the model
# rfe.fit(X, y)

# # Get the selected features
# selected_features = X.columns[rfe.support_]
# print("Selected Features:", selected_features)

# # Evaluate the model
# y_pred = rfe.predict(X)
# print(classification_report(y, y_pred))

# # Plot feature ranking
# plt.figure(figsize=(10, 6))
# plt.barh(X.columns, rfe.ranking_)
# plt.xlabel("Ranking")
# plt.ylabel("Features")
# plt.title("Feature Ranking with RFE")
# plt.show()


**Data Normalization**

In [9]:
selected_features=['Dst Port', 'Flow Byts/s',
       'Fwd IAT Mean', 'Bwd IAT Min', 'Fwd Pkts/s', 'Fwd Seg Size Avg',
       'Bwd Seg Size Avg', 'Subflow Bwd Byts', 'Init Fwd Win Byts',
       'Init Bwd Win Byts']

In [10]:
# %%time
# # list numeric columns
# # numeric_cols = network_data.select_dtypes(include=[np.number]).columns.tolist()

# numeric_cols = ['Dst Port', 'Flow Byts/s',
#        'Fwd IAT Mean', 'Bwd IAT Min', 'Fwd Pkts/s', 'Fwd Seg Size Avg',
#        'Bwd Seg Size Avg', 'Subflow Bwd Byts', 'Init Fwd Win Byts',
#        'Init Bwd Win Byts']

# print(numeric_cols)

In [11]:
selected_features_df = network_data[selected_features]
print(selected_features_df.head())

   Dst Port  Flow Byts/s  Fwd IAT Mean  Bwd IAT Min    Fwd Pkts/s  \
0       445     0.000000  1.294750e+05          0.0     11.585248   
1      3389  1929.697409  1.839667e+05     104244.0      6.039741   
2     10804     0.000000  4.600000e+01          0.0  43478.260870   
3       443   281.709029  1.974630e+06       1019.0      0.515007   
4     56057     0.000000  4.100000e+01          0.0  48780.487805   

   Fwd Seg Size Avg  Bwd Seg Size Avg  Subflow Bwd Byts  Init Fwd Win Byts  \
0               0.0             0.000                 0               8192   
1             146.8           215.875              1727               8192   
2               0.0             0.000                 0                258   
3             347.5           199.500             11970                255   
4               0.0             0.000                 0                257   

   Init Bwd Win Byts  
0               8192  
1              64000  
2                 -1  
3                440  
4

In [12]:
selected_features_df

,Dst Port,Flow Byts/s,Fwd IAT Mean,Bwd IAT Min,Fwd Pkts/s,Fwd Seg Size Avg,Bwd Seg Size Avg,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts
0,445,0.000000,1.294750e+05,0.0,11.585248,0.000000,0.000,0,8192,8192
1,3389,1929.697409,1.839667e+05,104244.0,6.039741,146.800000,215.875,1727,8192,64000
2,10804,0.000000,4.600000e+01,0.0,43478.260870,0.000000,0.000,0,258,-1
3,443,281.709029,1.974630e+06,1019.0,0.515007,347.500000,199.500,11970,255,440
4,56057,0.000000,4.100000e+01,0.0,48780.487805,0.000000,0.000,0,257,-1
...,...,...,...,...,...,...,...,...,...,...
905985,8080,42086.763480,2.690000e+02,51.0,277.495144,108.666667,32.250,129,8192,219
905986,8080,39045.739290,2.710000e+02,23.0,257.444435,108.666667,32.250,129,8192,219
905987,8080,0.000000,4.420000e+02,0.0,4524.886878,0.000000,0.000,0,2052,-1
905988,8080,48424.861640,2.735000e+02,19.0,319.284802,108.666667,32.250,129,8192,219


In [13]:
scaler = RobustScaler()

In [14]:
scaled_data = scaler.fit_transform(selected_features_df)

In [15]:
scaled_features_df = pd.DataFrame(scaled_data, columns=selected_features)
print(scaled_features_df.head())



   Dst Port  Flow Byts/s  Fwd IAT Mean  Bwd IAT Min  Fwd Pkts/s  \
0  0.045625    -0.022091      0.329560     0.000000   -0.033050   
1  0.413625     0.122627      0.471118   353.369492   -0.034693   
2  1.340500    -0.022091     -0.006669     0.000000   12.839837   
3  0.045375    -0.000964      5.122877     3.454237   -0.036329   
4  6.997125    -0.022091     -0.006682     0.000000   14.410120   

   Fwd Seg Size Avg  Bwd Seg Size Avg  Subflow Bwd Byts  Init Fwd Win Byts  \
0         -0.236196         -0.387960         -0.283784           0.230287   
1          1.114724          1.056020          4.383784           0.230287   
2         -0.236196         -0.387960         -0.283784          -0.067334   
3          2.961656          0.946488         32.067568          -0.067447   
4         -0.236196         -0.387960         -0.283784          -0.067372   

   Init Bwd Win Byts  
0          37.236364  
1         290.909091  
2          -0.004545  
3           2.000000  
4          -0

In [16]:
scaled_features_df

,Dst Port,Flow Byts/s,Fwd IAT Mean,Bwd IAT Min,Fwd Pkts/s,Fwd Seg Size Avg,Bwd Seg Size Avg,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts
0,0.045625,-0.022091,0.329560,0.000000,-0.033050,-0.236196,-0.387960,-0.283784,0.230287,37.236364
1,0.413625,0.122627,0.471118,353.369492,-0.034693,1.114724,1.056020,4.383784,0.230287,290.909091
2,1.340500,-0.022091,-0.006669,0.000000,12.839837,-0.236196,-0.387960,-0.283784,-0.067334,-0.004545
3,0.045375,-0.000964,5.122877,3.454237,-0.036329,2.961656,0.946488,32.067568,-0.067447,2.000000
4,6.997125,-0.022091,-0.006682,0.000000,14.410120,-0.236196,-0.387960,-0.283784,-0.067372,-0.004545
...,...,...,...,...,...,...,...,...,...,...
905985,1.000000,3.134226,-0.006089,0.172881,0.045700,0.763804,-0.172241,0.064865,0.230287,0.995455
905986,1.000000,2.906163,-0.006084,0.077966,0.039762,0.763804,-0.172241,0.064865,0.230287,0.995455
905987,1.000000,-0.022091,-0.005640,0.000000,1.303588,-0.236196,-0.387960,-0.283784,-0.000038,-0.004545
905988,1.000000,3.609554,-0.006078,0.064407,0.058077,0.763804,-0.172241,0.064865,0.230287,0.995455


In [17]:
scaled_features_df['Label'] = y.values

print(scaled_features_df.head())



   Dst Port  Flow Byts/s  Fwd IAT Mean  Bwd IAT Min  Fwd Pkts/s  \
0  0.045625    -0.022091      0.329560     0.000000   -0.033050   
1  0.413625     0.122627      0.471118   353.369492   -0.034693   
2  1.340500    -0.022091     -0.006669     0.000000   12.839837   
3  0.045375    -0.000964      5.122877     3.454237   -0.036329   
4  6.997125    -0.022091     -0.006682     0.000000   14.410120   

   Fwd Seg Size Avg  Bwd Seg Size Avg  Subflow Bwd Byts  Init Fwd Win Byts  \
0         -0.236196         -0.387960         -0.283784           0.230287   
1          1.114724          1.056020          4.383784           0.230287   
2         -0.236196         -0.387960         -0.283784          -0.067334   
3          2.961656          0.946488         32.067568          -0.067447   
4         -0.236196         -0.387960         -0.283784          -0.067372   

   Init Bwd Win Byts  Label  
0          37.236364      0  
1         290.909091      0  
2          -0.004545      0  
3       

In [18]:
scaled_features_df

,Dst Port,Flow Byts/s,Fwd IAT Mean,Bwd IAT Min,Fwd Pkts/s,Fwd Seg Size Avg,Bwd Seg Size Avg,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Label
0,0.045625,-0.022091,0.329560,0.000000,-0.033050,-0.236196,-0.387960,-0.283784,0.230287,37.236364,0
1,0.413625,0.122627,0.471118,353.369492,-0.034693,1.114724,1.056020,4.383784,0.230287,290.909091,0
2,1.340500,-0.022091,-0.006669,0.000000,12.839837,-0.236196,-0.387960,-0.283784,-0.067334,-0.004545,0
3,0.045375,-0.000964,5.122877,3.454237,-0.036329,2.961656,0.946488,32.067568,-0.067447,2.000000,0
4,6.997125,-0.022091,-0.006682,0.000000,14.410120,-0.236196,-0.387960,-0.283784,-0.067372,-0.004545,0
...,...,...,...,...,...,...,...,...,...,...,...
905985,1.000000,3.134226,-0.006089,0.172881,0.045700,0.763804,-0.172241,0.064865,0.230287,0.995455,1
905986,1.000000,2.906163,-0.006084,0.077966,0.039762,0.763804,-0.172241,0.064865,0.230287,0.995455,1
905987,1.000000,-0.022091,-0.005640,0.000000,1.303588,-0.236196,-0.387960,-0.283784,-0.000038,-0.004545,1
905988,1.000000,3.609554,-0.006078,0.064407,0.058077,0.763804,-0.172241,0.064865,0.230287,0.995455,1


**Split Data**

In [19]:
X=scaled_features_df.drop(["Label"], axis=1)
y=scaled_features_df["Label"]

# split the data for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state =42, shuffle=True)

# K-fold
# kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [20]:
from sklearn.ensemble import RandomForestClassifier                   # Apply  RandomForestClassifier.
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_classifier.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

Accuracy: 0.9633881168666321

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.96     45424
           1       0.98      0.95      0.96     45175

    accuracy                           0.96     90599
   macro avg       0.96      0.96      0.96     90599
weighted avg       0.96      0.96      0.96     90599


Confusion Matrix:
[[44447   977]
 [ 2340 42835]]

Feature Importance:
             feature  importance
5   Fwd Seg Size Avg    0.189353
0           Dst Port    0.163519
3        Bwd IAT Min    0.109658
8  Init Fwd Win Byts    0.108062
6   Bwd Seg Size Avg    0.095266
9  Init Bwd Win Byts    0.089831
2       Fwd IAT Mean    0.065562
1        Flow Byts/s    0.064798
4         Fwd Pkts/s    0.060900
7   Subflow Bwd Byts    0.053050


In [21]:
from xgboost import XGBClassifier                       # Apply XGBClassifier.
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize the XGBoost Classifier
xgb_classifier = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Train the model
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': xgb_classifier.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

Accuracy: 0.9684212850031457

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     45424
           1       0.99      0.94      0.97     45175

    accuracy                           0.97     90599
   macro avg       0.97      0.97      0.97     90599
weighted avg       0.97      0.97      0.97     90599


Confusion Matrix:
[[45173   251]
 [ 2610 42565]]

Feature Importance:
             feature  importance
5   Fwd Seg Size Avg    0.497295
9  Init Bwd Win Byts    0.112096
0           Dst Port    0.084622
6   Bwd Seg Size Avg    0.079235
8  Init Fwd Win Byts    0.076094
3        Bwd IAT Min    0.071524
4         Fwd Pkts/s    0.050340
2       Fwd IAT Mean    0.016962
7   Subflow Bwd Byts    0.007287
1        Flow Byts/s    0.004544


In [24]:
!pip install catboost                                 # Apply CatBoostClassifier.
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize the CatBoost Classifier
catboost_classifier = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=5,
    loss_function='Logloss',
    random_seed=42,
    verbose=False
)

# Train the model
catboost_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = catboost_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': catboost_classifier.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 3.0 MB/s eta 0:00:00
Accuracy: 0.9655846091016457

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.99      0.97     45424
           1       0.99      0.94      0.96     45175

    accuracy                           0.97     90599
   macro avg       0.97      0.97      0.97     90599
weighted avg       0.97      0.97      0.97     90599


Confusion Matrix:
[[44955   469]
 [ 2649 42526]]

Feature Importance:
             feature  importance
0           Dst Port   26.826503
8  Init Fwd Win Byts   26.134893
5   Fwd Seg Size Avg   21.030266
9  Init Bwd Win Byts    9.972481
3        Bwd IAT Min    4.767395
4         Fwd Pkts/s    4.097854
2       Fwd IAT Mean    3.046405
6   Bwd Seg Size Avg    2.403196
1        Flow Byts/s    0.902796
7   Subflow Bwd Byts    0.818211


In [25]:
from sklearn.ensemble import AdaBoostClassifier           # Apply AdaBoostClassifier.
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize the AdaBoost Classifier
# We'll use a decision tree as the base estimator
adaboost_classifier = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=1.0,
    random_state=42
)

# Train the model
adaboost_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = adaboost_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': adaboost_classifier.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

Accuracy: 0.9583990993278072

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96     45424
           1       0.98      0.94      0.96     45175

    accuracy                           0.96     90599
   macro avg       0.96      0.96      0.96     90599
weighted avg       0.96      0.96      0.96     90599


Confusion Matrix:
[[44417  1007]
 [ 2762 42413]]

Feature Importance:
             feature  importance
8  Init Fwd Win Byts        0.32
9  Init Bwd Win Byts        0.13
0           Dst Port        0.12
5   Fwd Seg Size Avg        0.11
2       Fwd IAT Mean        0.10
4         Fwd Pkts/s        0.06
7   Subflow Bwd Byts        0.06
6   Bwd Seg Size Avg        0.05
3        Bwd IAT Min        0.03
1        Flow Byts/s        0.02


In [26]:
from lightgbm import LGBMClassifier                    # Apply  LGBMClassifier.
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize the LightGBM Classifier
lgbm_classifier = LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)

# Train the model
lgbm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lgbm_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgbm_classifier.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 407820, number of negative: 407571
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021773 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2489
[LightGBM] [Info] Number of data points in the train set: 815391, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500153 -> initscore=0.000611
[LightGBM] [Info] Start training from score 0.000611
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

In [27]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier  # Comprare with different models.
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def train_and_evaluate(model, X_train, X_test, y_train, y_test, model_name):
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)

    print(f"\n{model_name} Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 5 Important Features:")
    print(feature_importance.head())

    return accuracy, feature_importance

# Initialize models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, use_label_encoder=False, eval_metric='logloss')
ada_model = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1), n_estimators=100, learning_rate=1.0, random_state=42)
lgbm_model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# List of models
models = [
    (rf_model, "Random Forest"),
    (xgb_model, "XGBoost"),
    (ada_model, "AdaBoost"),
    (lgbm_model, "LightGBM")
]

# Train and evaluate all models
results = {}
for model, name in models:
    accuracy, feature_importance = train_and_evaluate(model, X_train, X_test, y_train, y_test, name)
    results[name] = {'accuracy': accuracy, 'feature_importance': feature_importance}

# Compare accuracies
print("\nAccuracy Comparison:")
for name, result in results.items():
    print(f"{name}: {result['accuracy']:.4f}")

# Compare top 3 important features across models
print("\nTop 3 Important Features Comparison:")
for name, result in results.items():
    top_3_features = result['feature_importance'].head(3)['feature'].tolist()
    print(f"{name}: {', '.join(top_3_features)}")


Random Forest Results:
Accuracy: 0.9634

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.96     45424
           1       0.98      0.95      0.96     45175

    accuracy                           0.96     90599
   macro avg       0.96      0.96      0.96     90599
weighted avg       0.96      0.96      0.96     90599

Confusion Matrix:
[[44447   977]
 [ 2340 42835]]

Top 5 Important Features:
             feature  importance
5   Fwd Seg Size Avg    0.189353
0           Dst Port    0.163519
3        Bwd IAT Min    0.109658
8  Init Fwd Win Byts    0.108062
6   Bwd Seg Size Avg    0.095266

XGBoost Results:
Accuracy: 0.9684

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     45424
           1       0.99      0.94      0.97     45175

    accuracy                           0.97     90599
   macro avg       0.97      0.97      0.97     90599
wei